In [26]:
from operator import itemgetter
import itertools
from IPython.display import HTML

import pandas as pd
from Mike.old_sa import SubstrateAnalyzer
#from pymatgen.analysis.interfaces.substrate_analyzer import SubstrateAnalyzer
from pymatgen.ext.matproj import MPRester
from tqdm import tqdm
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import pymatgen.core as mg
from pymatgen.core.lattice import Lattice

from pymatgen.io.ase import AseAtomsAdaptor
from ase.db import connect
import glob
import os
import json
from ase import io
from ase.data import chemical_symbols, atomic_numbers

from bokeh.models import ColumnDataSource, Label, LabelSet, Range1d, HoverTool
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import row

import numpy as np
import matplotlib.pyplot as plt
mpr = MPRester("NGpMC4M2qzg2ZpPEX")

import html5lib
import bs4

from pymatgen.core.composition import Composition

import pickle
dbs = [connect(os.path.abspath('mp/mp_oxides.db')), connect(os.path.abspath('oqmd/oqmd_oxides.db'))]
all_BCC = pickle.load(open("Mike/all_BCC.pickle", "rb"))
all_FCC = pickle.load(open("Mike/all_FCC.pickle", "rb"))

In [2]:
#df = pd.read_html("Mike/MCIAhtmlNew/BCC-Nb(0.33)+Ti(0.33)+Zr(0.34).html")

In [1]:
# # f_energies = []
# # mcias = []
# # labels = []
# plt.figure(figsize = (14, 10))
# for row in df[0].iterrows():
#     plt.scatter(row[1][4], row[1][10])
#     plt.annotate(row[1][2], # this is the text
#                  (row[1][4], row[1][10]), # these are the coordinates to position the label
#                  textcoords="offset points", # how to position the text
#                  xytext=(0,5), # distance from text to points (x,y)
#                  ha='center')
# plt.ylabel("MCIA")
# plt.xlabel("Formation energy per atom (eV/atom)")

In [2]:
def make_query_string_from_elements(query_elements):

    symbol_mask = [False for symbol in chemical_symbols]

    for symbol in query_elements: # marks the ones we want
        symbol_mask[atomic_numbers[symbol]] = True

    query_string = ''
    for z, sb in enumerate(symbol_mask):
        if sb:
            query_string += chemical_symbols[z] +'>=0,'
        else:
            query_string += chemical_symbols[z] +'=0,'

    return query_string


def make_alloy_string_from_elements(query_elements):
    qes = sorted(query_elements, key=lambda el: atomic_numbers[el])

    alloy_name = ''
    for el in qes:
        alloy_name+=el

    return alloy_name

In [3]:
def make_spec_query_string_from_elements(query_elements, amounts):
    
    if len(query_elements) != len(amounts):
        print("Invalid inputs!")

    symbol_mask = [False for symbol in chemical_symbols]

    for symbol in query_elements: # marks the ones we want
        symbol_mask[atomic_numbers[symbol]] = True

    query_string = ''
    i = 0
    for z, sb in enumerate(symbol_mask):
        if sb:
            query_string += chemical_symbols[z] +'='+ str(amounts[i]) + ','
            i += 1
        else:
            query_string += chemical_symbols[z] +'=0,'

    return query_string

In [4]:
def groupby_itemkey(iterable, item):
    """groupby keyed on (and pre-sorted by) itemgetter(item)."""
    itemkey = itemgetter(item)
    return itertools.groupby(sorted(iterable, key=itemkey), itemkey)

In [5]:
def MCIA(f, s, conventional = False):
    all_matches = []
    sa = SubstrateAnalyzer()

    if conventional:
        film = SpacegroupAnalyzer(f).get_conventional_standard_structure()
        substrate = SpacegroupAnalyzer(s).get_conventional_standard_structure()
    else:
        film = f
        substrate = s

    # Calculate all matches and group by substrate orientation
    matches_by_orient = groupby_itemkey(
        sa.calculate(film, substrate, lowest = True),
        "sub_miller")

    # Find the lowest area match for each substrate orientation
    lowest_matches = [min(g, key=itemgetter("match_area"))
                      for k, g in matches_by_orient]

    for match in lowest_matches:
        print(match)
        db_entry = {
            "sub_form": substrate.composition.reduced_formula,
            "orient": " ".join(map(str, match["sub_miller"])),
            "film_form": film.composition.reduced_formula,
            "film_orient": " ".join(map(str, match["film_miller"])),
            "area": match["match_area"],
        }
        all_matches.append(db_entry)

    df = pd.DataFrame(all_matches)
    #df.set_index("sub_id", inplace=True)
    return df.sort_values("area")

In [7]:
query_elements = ['Ni', 'O']
other_query_elements = ['Ni']
my_qs = make_query_string_from_elements(query_elements)
other_my_qs = make_spec_query_string_from_elements(other_query_elements, [1])
alloy_name = make_alloy_string_from_elements(query_elements)

In [31]:
# howmany = 0
# all_atoms = []
# for db in dbs:
#     for row in db.select(pretty_formula = 'NiO', e_above_hull = 0):
#         print(row.key_value_pairs)
#         print()
#         atoms = row.toatoms()
#         all_atoms.append(atoms)
#         howmany +=1 
# print(howmany)

In [5]:
def get_equiv_alloy_volume(atoms):
    symbols = atoms.get_chemical_symbols()

    volume = 0

    for sym in symbols:
        if sym != 'O':
            volume+=metallic_atomic_volumes[sym]
    return volume

In [6]:
def equiv_alloy_volume(atoms, per_cell_volume):
    symbols = atoms.get_chemical_symbols()
    volume = 0
    for sym in symbols:
        if sym != 'O':
            volume+=per_cell_volume
    return volume

In [7]:
def get_substrate(lattice_param, structure):
    if structure == "BCC":
        return mg.Structure(Lattice.cubic(lattice_param), ["Cu", "Cu"], [[0, 0, 0], [0.5, 0.5, 0.5]])
    elif structure == "FCC":
        return mg.Structure(Lattice.cubic(lattice_param), ["Cu", "Cu", "Cu", "Cu"], [[0, 0, 0], [0.5, 0.5, 0], [0.5, 0, 0.5], [0, 0.5, 0.5]])

In [29]:
def get_films(metals, per_cell_volume, e_above_hull_lim):
    howmany = 0
    my_qs = make_query_string_from_elements(['O'] + metals) + "e_above_hull<=" + str(e_above_hull_lim)
    substrates = []
    for db in dbs:
        for row in db.select(my_qs):
            atoms = row.toatoms()
            if len(set(atoms.get_chemical_symbols())) == 1:
                continue
            v_mox_ratio = atoms.cell.volume / equiv_alloy_volume(atoms, per_cell_volume)
            try:
                substrates.append({"structure": AseAtomsAdaptor.get_structure(atoms), 
                                   "atoms": atoms,
                                   "source": row['material_id'],
                                   "natoms": row['natoms'],
                                   "spg_num": row['spg_number'],
                                   "formula": row['pretty_formula'],
                                   "ox_at_%": 100*Composition(row['pretty_formula']).get_atomic_fraction("O"),
                                   "e_above_hull": row['e_above_hull'],
                                   "formation_energy_per_atom": row["formation_energy_per_atom"],
                                   "space group": row["spg_symbol"],
                                   "v_mox_ratio": v_mox_ratio,
                                   "ICSD IDs": process_ICSD(row.data.icsd_ids),
                                   "band gap": row["band_gap"]})
            except:
                print('Tabulation failed for:', row)
            howmany +=1
    return substrates

In [11]:
from itertools import compress
def sort_through_dupes(films):
    want = np.ones(len(films), dtype = bool)
    for i in range(len(films)):
        for j in range(i+1, len(films)):
            if (films[i]["formula"] == films[j]["formula"]) and (films[i]["natoms"] == films[j]["natoms"]) and (films[i]["spg_num"] == films[j]["spg_num"]):
                if films[i]["e_above_hull"] < films[j]["e_above_hull"]:
                    want[j] = False
                else:
                    want[i] = False
    return list(compress(films, want))

In [12]:
def print_mat_ids(films):
    for i in range(len(films)):
        print(films[i]["source"])

In [13]:
def process_ICSD(ICSDs):
    def format_int_list(ilist):
        if len(ilist) > 0:
            out_string = '{'
            for iint in ilist[0:-1]:
                out_string+='%i, ' %iint
            out_string+= '%i}'% ilist[-1]
        else:
            out_string = ''
        return out_string

    return format_int_list(sorted(ICSDs))

In [14]:
from pymatgen.analysis.elasticity.strain import Deformation
from pymatgen.core.surface import (SlabGenerator,
                                   get_symmetrically_distinct_miller_indices)
def misfit_strain(film, match, norm = True):
    def fast_norm(a):
        return np.sqrt(np.dot(a, a))
    # need film to be conventional standard structure
    struc = SlabGenerator(film, match['film_miller'], 20, 15,
                              primitive=True).get_slab().oriented_unit_cell

    # Generate 3D lattice vectors for film super lattice
    film_matrix = list(match['film_sl_vecs'])
    film_matrix.append(np.cross(film_matrix[0], film_matrix[1]))

    # Generate 3D lattice vectors for substrate super lattice
    # Out of plane substrate super lattice has to be same length as
    # Film out of plane vector to ensure no extra deformation in that
    # direction
    substrate_matrix = list(match['sub_sl_vecs'])
    temp_sub = np.cross(substrate_matrix[0], substrate_matrix[1])
    temp_sub = temp_sub * fast_norm(film_matrix[2]) / fast_norm(temp_sub)
    substrate_matrix.append(temp_sub)

    transform_matrix = np.transpose(np.linalg.solve(film_matrix,
                                                    substrate_matrix))
    dfm = Deformation(transform_matrix)

    strain = dfm.green_lagrange_strain.convert_to_ieee(struc, initial_fit=False)

    if norm:
        return (np.sum(np.power(np.array(strain), 2))) ** 0.5
    else:
        return strain.von_mises_strain

In [35]:
# conventional substrate, primitive films
def analyze_alloy(metals, composition, structure, e_above_hull_lim = 0.015, no_dupes = True):
    # deal with inputs
    if len(metals) != len(composition):
        print("Inputs do not match!")
        return None
    if round(sum(composition), 6) != 1:
        print("Composition does not add to 1!")
        return None
    parameter = 0
    if structure == "BCC":
        using = all_BCC
        factor = 2
    elif structure == "FCC":
        using = all_FCC
        factor = 4
    for i, metal in enumerate(metals):
        if using.get(metal) is None:
            print("Do not have lattice parameter for:", metal)
            return None
        parameter += using[metal]["val"] * composition[i]
    substrate = SpacegroupAnalyzer(get_substrate(parameter, structure)).get_conventional_standard_structure()
    per_cell_volume = substrate.volume / factor
    films = get_films(metals, per_cell_volume, e_above_hull_lim)
    if no_dupes:
        films = sort_through_dupes(films)
    # !!! actual analyzer part !!!
    all_matches = []
    sa = SubstrateAnalyzer()
    # Calculate all matches and group by substrate orientation
    for f in tqdm(films):
        film = f["structure"]
        matches_by_orient = groupby_itemkey(
            sa.calculate(film, substrate, lowest = True),
            "sub_miller")

        # Find the lowest area match for each substrate orientation
        lowest_matches = [min(g, key=itemgetter("match_area"))
                          for k, g in matches_by_orient]

        going = True
        for match in lowest_matches:
            try:
                db_entry = {
                    "material ID": f["source"],
                    "oxide": film.composition.reduced_formula,
                    "e_above_hull (eV)": f['e_above_hull'],
                    "form. energy per atom (eV/atom)": f["formation_energy_per_atom"],
                    "space-group": f["space group"],
                    "v_mox_ratio": f["v_mox_ratio"],
                    "atomic % of O in oxide": round(f["ox_at_%"], 2),
                    "band gap": f["band gap"],
                    "alloy orientation": " ".join(map(str, match["sub_miller"])),
                    "oxide orientation": " ".join(map(str, match["film_miller"])),
                    "misfit strain (norm)": misfit_strain(film, match, norm = True),
                    "min. coincident area": match["match_area"],
                    "similar ICSD structure IDs": f["ICSD IDs"],
                }
                all_matches.append(db_entry)
            except:
                print(f["source"], "with composition", film.composition.reduced_formula, "failed")
#             if f["source"] == "oqmd-2575" and going:
#                 going = False
#                 gen_interface(parameter, structure.lower(), match["sub_sl_vecs"], match["film_sl_vecs"], f["atoms"])
    df = pd.DataFrame(all_matches)
    df = df.sort_values("min. coincident area")
    print("Calculated parameter is:", parameter)
    print("Found", len(films), "film candidates.")
    return parameter, df

In [19]:
def display_data(df, num_display = None, top_unique = None):
    #pd.set_option("display.max_rows", None, "display.max_columns", None)
    if top_unique is not None:
        new_df = df.iloc[:top_unique].append((df.iloc[top_unique:]).drop_duplicates(subset=['material ID']))
    else:
        new_df = df.copy(deep = True)
    if num_display is None or len(df) < num_display:
        return new_df
    else:
        return new_df.head(num_display)

In [20]:
def formatting(x):
    if "mp" in x: 
        return '<a href="https://materialsproject.org/materials/{0}" target="_blank">{0}</a>'.format(x)
    else: 
        return '<a href="http://oqmd.org/materials/entry/{0}" target="_blank">{1}</a>'.format(x[5:], x)

In [21]:
def make_sortable(name_of_file):
    with open(name_of_file, 'r') as file:
        # read a list of lines into data
        data = file.readlines()

    with open("MCIAheading.txt", 'r') as headingfile:
        # read a list of lines into data
        headingdata = headingfile.readlines()

    for i in range(len(headingdata)):
        data[i] = headingdata[i]

    with open(name_of_file, 'w') as file:
        file.writelines(data)

    f1 = open(name_of_file, 'a+')
    f2 = open("MCIAscript.txt", 'r')

    f1.write(f2.read())

    f1.close()
    f2.close()

In [22]:
from bokeh.layouts import gridplot
from bokeh.models import BooleanFilter, CDSView, ColumnDataSource
from bokeh.plotting import figure, output_file, show
from bokeh.models.tools import *

def add_plots(df, name_of_pfile, name_of_file, append_string):

    output_file(name_of_pfile)

    source = ColumnDataSource(data=dict(form=list(df["form. energy per atom (eV/atom)"]),
                                        mcia=list(df["min. coincident area"]),
                                        strain=list(df["misfit strain (norm)"]),
                                        names=list(df["oxide"]),
                                        mp_ids=list(df["material ID"]),
                                        ox_orients=list(df["oxide orientation"])))
    
    hover = HoverTool(tooltips=[
        ("mat_id", "@mp_ids"),
        ("oxide orientation", "@ox_orients"),
        ('mcia', '@mcia'),
        ('misfit strain', '@strain'),
        ('formation energy', '@form'),
        ])

    # create a view of the source for one plot to use
    view = CDSView(source=source)

    TOOLS = [BoxZoomTool(), PanTool(), SaveTool(), ResetTool(), hover]

    # create a new plot and add a renderer
    left = figure(tools=TOOLS, title='MCIA vs Formation Energy Per Atom')
    left.circle(x='form', y='mcia', size=8, hover_color="deeppink", source=source)
    left.xaxis[0].axis_label = 'Formation energy per atom (eV/atom)'
    left.yaxis[0].axis_label = 'MCIA'
    
    labels = LabelSet(x='form', y='mcia', text='names',
                  x_offset=3, y_offset=3, source=source, render_mode='canvas', text_font_size="8pt")

    left.add_layout(labels)
   
    # create another new plot, add a renderer that uses the view of the data source
    right = figure(tools=TOOLS, title='Misfit Strain vs Formation Energy Per Atom')
    right.circle(x='form', y='strain', size=8, hover_color="deeppink", source=source, view=view)
    right.xaxis[0].axis_label = 'Formation energy per atom (eV/atom)'
    right.yaxis[0].axis_label = 'Misfit Strain'
    
    labels = LabelSet(x='form', y='strain', text='names',
                  x_offset=3, y_offset=3, source=source, render_mode='canvas', text_font_size="8pt")

    right.add_layout(labels)

    p = gridplot([[left, right]])

    show(p)
    
    f1 = open(name_of_pfile, 'a+')
    f2 = open(name_of_file, 'r')
    
    f1.write("<br>")
    
    f1.write(append_string)
    
    f1.write("<br><br>")

    f1.write(f2.read())

    f1.close()
    f2.close()

In [23]:
def auto_comp(metals):
    return np.ones(len(metals)) / len(metals)

In [20]:
from ase import io
from ase.build.supercells import make_supercell
from ase.build import cut, bulk

import numpy as np
def gen_interface(lattice_parameter, latt, sub_sl_vecs, film_sl_vecs, film_uc):

    element = 'Nb'
    metal_lattice_parameter_a = lattice_parameter

    sub_thickness = 15
    film_thickness = 15

    interface_gap = 1.5 #Ang

    sub_uc = bulk(element, latt, a=metal_lattice_parameter_a, cubic = True)

    ##################

    #sub_vecs                 = np.array([[ 3.37024467e+00,  0.00000000e+00, 0.00000000e+00], [ 0.00000000e+00, 3.37024467e+00, 0.00000000e+00]])
    #substrate_transformation = np.array([[ 1., 2.], [0., 5.]])
    #sub_miller               = np.array([0,0,1]) #np.array([ 1, 0, 0])

    #, 'film_miller': (0, 0, 1),
    mults = sub_uc.cell.scaled_positions(sub_sl_vecs)
    normal = np.cross(sub_sl_vecs[0], sub_sl_vecs[1])
    normal = normal/np.linalg.norm(normal)
    normal_mult = sub_uc.cell.scaled_positions(sub_thickness*normal)
    print('sub cuts\n', mults)

    a = mults[0]
    b = mults[1]
    c = normal_mult

    substrate = cut(sub_uc, a= a, b = b, c = c)

    print('subtrate top', substrate.cell[2])

    ########################


    #film_vecs           = np.array([[ 3.560158 , 0. , -1.3151887 ], [-0.48585521, 3.52684983, -1.3151887 ]])
    #film_transformation = np.array([[2., 0.], [0., 2.]]),
    #film_miller         = np.array([1, 0, 0])

    mults = film_uc.cell.scaled_positions(film_sl_vecs)
    normal = np.cross(film_sl_vecs[0], film_sl_vecs[1])
    #mults = film_uc.cell.scaled_positions(sub_sl_vecs)
    #normal = np.cross(sub_sl_vecs[0], sub_sl_vecs[1])

    normal = normal/np.linalg.norm(normal)
    normal_mult = film_uc.cell.scaled_positions(film_thickness*normal)
    print('film cuts\n', mults)

    a = mults[0]
    b = mults[1]
    c = normal_mult

    film = cut(film_uc, a= a, b = b, c = c, tolerance=0.1)

    ###############
    film_matrix = list(film_sl_vecs)
    film_matrix.append(np.cross(film_matrix[0], film_matrix[1]))

    # Generate 3D lattice vectors for substrate super lattice
    # Out of plane substrate super lattice has to be same length as
    # Film out of plane vector to ensure no extra deformation in that
    # direction
    substrate_matrix = list(sub_sl_vecs)
    temp_sub = np.cross(substrate_matrix[0], substrate_matrix[1])
    temp_sub = temp_sub * np.linalg.norm(film_matrix[2]) / np.linalg.norm(temp_sub)
    substrate_matrix.append(temp_sub)

    transform_matrix = np.transpose(np.linalg.solve(film_matrix, substrate_matrix))
    print('tmat',transform_matrix)
    print('|tmat|',np.linalg.det(transform_matrix))

    #########
    rpos  = film.get_positions() @ transform_matrix.T
    film.set_positions(rpos)
    rcell = film.get_cell() @ transform_matrix.T
    film.set_cell(rcell)
    ##########
    cv = substrate.cell[2]
    bottom = cv + interface_gap*cv/np.linalg.norm(cv)
    top = bottom+film.cell[2]
    newcell = substrate.get_cell()
    newcell[2] = top

    f = film.copy()
    f.translate(bottom)
    interface = substrate.copy() + f

    interface.set_cell(newcell)

    io.write('interface.POSCAR', interface)

    ######
    from ase.visualize import view
    parts = [sub_uc, substrate, interface, film, film_uc]
    view(parts)

In [1]:
from ase import io
from ase.build.supercells import make_supercell

import numpy as np
def gen_interface(sub_uc, film_uc, latt, sub_sl_vecs, film_sl_vecs):
    sub_thickness = 15
    film_thickness = 15

    mults = sub_uc.cell.scaled_positions(sub_sl_vecs)
    normal = np.cross(sub_sl_vecs[0], sub_sl_vecs[1])
    normal = normal/np.linalg.norm(normal)
    normal_mult = sub_uc.cell.scaled_positions(sub_thickness*normal)
    print('sub cuts\n', mults)

    a = mults[0]
    b = mults[1]
    c = normal_mult

    substrate = cut(sub_uc, a= a, b = b, c = c)

    print('subtrate top', substrate.cell[2])

    mults = film_uc.cell.scaled_positions(film_sl_vecs)
    normal = np.cross(film_sl_vecs[0], film_sl_vecs[1])
   
    normal = normal/np.linalg.norm(normal)
    normal_mult = film_uc.cell.scaled_positions(film_thickness*normal)
    print('film cuts\n', mults)

    a = mults[0]
    b = mults[1]
    c = normal_mult

    film = cut(film_uc, a= a, b = b, c = c, tolerance=0.1)

    ###############
    film_matrix = list(film_sl_vecs)
    film_matrix.append(np.cross(film_matrix[0], film_matrix[1]))

    # Generate 3D lattice vectors for substrate super lattice
    # Out of plane substrate super lattice has to be same length as
    # Film out of plane vector to ensure no extra deformation in that
    # direction
    substrate_matrix = list(sub_sl_vecs)
    temp_sub = np.cross(substrate_matrix[0], substrate_matrix[1])
    temp_sub = temp_sub * np.linalg.norm(film_matrix[2]) / np.linalg.norm(temp_sub)
    substrate_matrix.append(temp_sub)

    transform_matrix = np.transpose(np.linalg.solve(film_matrix, substrate_matrix))
    print('tmat',transform_matrix)
    print('|tmat|',np.linalg.det(transform_matrix))

    #########
    rpos  = film.get_positions() @ transform_matrix.T
    film.set_positions(rpos)
    rcell = film.get_cell() @ transform_matrix.T
    film.set_cell(rcell)
    ##########
    cv = substrate.cell[2]
    bottom = cv + interface_gap*cv/np.linalg.norm(cv)
    top = bottom+film.cell[2]
    newcell = substrate.get_cell()
    newcell[2] = top

    f = film.copy()
    f.translate(bottom)
    interface = substrate.copy() + f

    interface.set_cell(newcell)

    io.write('interface.POSCAR', interface)

    ######
    from ase.visualize import view
    parts = [sub_uc, substrate, interface, film, film_uc]
    view(parts)

In [36]:
# df stored
#METALS = ["Al", "Co", "Cr", "Fe", "Mn", "Ni", "Ti"]
#METALS = ["Co", "Cr", "Fe", "Mn", "Ni", "Ti"]
#METALS = ["Hf", "Nb", "Ta", "Ti", "Zr", "Al"]
# BCC Hf_0.198_Nb_0.198_Ta_0.198_Ti_0.198_Zr_0.198_Al_0.01
METALS = ["Fe", "Co", "Ni"]
#METALS = ["Cr", "Al", "Fe", "Co", "Ni"]

#COMPS = [0.198, 0.198, 0.198, 0.198, 0.198, 0.01]
#COMPS = auto_comp(METALS)
COMPS = [0.33, 0.33, 0.34]
#COMPS = [0.10, 0.10, 0.26, 0.27, 0.27]

#print(COMPS)
E_HULL = 0.015
STRUCTURE = "BCC"
lattice_param, df = analyze_alloy(METALS, COMPS, STRUCTURE, e_above_hull_lim = E_HULL)

  0%|                                                                                           | 0/48 [00:00<?, ?it/s]

Tabulation failed for: <AtomsRow: formula=Fe4O6, keys=e_above_hull,formation_energy_per_atom,material_id,spg_number,spg_symbol,pretty_formula>


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:48<00:00,  1.01s/it]

Calculated parameter is: 2.8255107360359517
Found 48 film candidates.


In [37]:
dfcopy = display_data(df, top_unique = 5)
dfcopy2 = display_data(df, top_unique = 5)

In [38]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [39]:
# create html out of the df
calc_param = lattice_param
dfcopy['material ID'] = dfcopy['material ID'].apply(formatting)
specific_part = STRUCTURE + "-" + "+".join("{0}({1})".format(i, round(j, 3)) for i, j in zip(METALS, COMPS)) + "e_lim=" + str(E_HULL) + ".html"
name_of_file = 'Mike/MCIAhtmlNew/' + specific_part
name_of_pfile = 'Mike/MCIAwPlots/' + specific_part
dfcopy.to_html(open(name_of_file, 'w'), 
                    justify = "center", 
                    table_id = "myTable2",
                    render_links=True, 
                    escape = False)
make_sortable(name_of_file)
append_string = STRUCTURE + "-" + "_".join("{0}({1})".format(i, round(j, 3)) for i, j in zip(METALS, COMPS)) + ": a = " + str(round(calc_param, 4)) + " Ang"
add_plots(dfcopy2, name_of_pfile, name_of_file, append_string)

In [39]:
# showing top 5 and only unique films afterwards
new = display_data(df, top_unique = 5)
display(new)

,material ID,oxide,e_above_hull (eV),form. energy / atom (eV/atom),space-group,v_mox_ratio,band gap,alloy orientation,oxide orientation,min. coincident area,similar ICSD structure IDs
17,mp-1215204,ZrTi2O,0.0,-1.596376,Cmmm,1.045165,0.0000,1 1 1,0 0 1,39.581032,
33,oqmd-3278,ZrO2,0.0,-3.632982,P21/c,1.875461,3.8430,1 0 0,1 1 1,44.999565,{15983}
46,oqmd-10207,TiO,0.0,-2.742762,C2/m,1.114467,0.0000,1 1 0,0 1 0,48.194680,{56694}
16,mp-1215204,ZrTi2O,0.0,-1.596376,Cmmm,1.045165,0.0000,1 1 0,1 -1 1,48.708924,
57,oqmd-87089,Ti2O,0.0,-1.986373,P-3m1,0.942407,0.0000,1 1 0,1 1 0,48.885153,{99784}
31,oqmd-2921,NbO,0.0,-2.154957,Pm-3m,1.324574,0.0000,1 1 0,1 1 0,50.774556,{14338}
27,oqmd-2575,TiO2,0.0,-3.242641,I41/amd,1.796183,1.9650,1 0 0,0 0 1,57.617785,{9852}
47,oqmd-10207,TiO,0.0,-2.742762,C2/m,1.114467,0.0000,1 1 1,0 0 1,76.931529,{56694}
44,oqmd-5214,Zr3O,0.0,-1.478765,R-3c,1.255407,0.0000,1 1 1,0 0 1,82.950186,{27023}
20,mp-1215377,Zr4O,0.0,-1.240729,R-3,1.262439,0.0000,1 1 1,1 1 1,83.250911,


In [160]:
df

,film_id,film_formula,e_above_hull (eV),form. energy / atom (eV/atom),space-group,v_mox_ratio,sub_orient,film_orient,area
147,oqmd-8206,NiO,0.0,-1.241513,R-3m,1.425928,1 0 0,1 1 0,8.728304
148,oqmd-8206,NiO,0.0,-1.241513,R-3m,1.425928,1 1 0,1 0 1,12.336859
46,mp-19009,NiO,0.0,-1.218085,Fm-3m,1.465720,1 0 0,1 0 -1,17.779849
154,oqmd-10207,TiO,0.0,-2.742762,C2/m,1.668168,1 1 0,0 1 0,24.097340
47,mp-19009,NiO,0.0,-1.218085,Fm-3m,1.465720,1 1 0,1 1 0,25.131306
...,...,...,...,...,...,...,...,...,...
5,mp-705564,Co23O32,0.0,-1.283408,P1,1.908738,1 1 1,1 0 0,304.027137
3,mp-705564,Co23O32,0.0,-1.283408,P1,1.908738,1 0 0,1 0 0,304.027137
66,mp-1278696,TiFeO3,0.0,-2.729715,P-1,2.142410,1 1 1,0 0 1,308.445562
133,oqmd-5667,Ti3Co3O,0.0,-1.048708,Fd-3m,1.121487,1 0 0,1 0 -1,309.388921


In [19]:
howmany = 0
#metals = ["Al", "Ti", "Cr", "Mn", "Fe", "Ni"]
metals = ["Ni"]
my_qs = make_query_string_from_elements(['O'] + metals) + "e_above_hull<0.15"
for db in dbs:
    for row in db.select(my_qs):
        atoms = row.toatoms()
        if len(set(atoms.get_chemical_symbols())) == 1:
            continue
        print(row.key_value_pairs)
        howmany +=1

{'material_id': 'mp-705519', 'pretty_formula': 'Ni15O16', 'spg_symprec': 0.1, 'spg_source': 'spglib', 'spg_symbol': 'Im-3m', 'spg_number': 229, 'point_group': '# m-3m', 'spg_crystal_system': 'cubic', 'spg_hall': '-I 4 2 3', 'e_above_hull': 0.03822233306450862, 'formation_energy_per_atom': -1.1411743664516132, 'energy_per_atom': -4.907704246451613, 'volume': 296.61752560376635, 'total_magnetization': 27.9998899, 'band_gap': 0.0}
{'material_id': 'mp-656887', 'pretty_formula': 'Ni3O4', 'spg_symprec': 0.1, 'spg_source': 'spglib', 'spg_symbol': 'Cmmm', 'spg_number': 65, 'point_group': '# mmm', 'spg_crystal_system': 'orthorhombic', 'spg_hall': '-C 2 2', 'e_above_hull': 0, 'formation_energy_per_atom': -1.0467500692857143, 'energy_per_atom': -4.8697863635714285, 'volume': 142.36088395782411, 'total_magnetization': 1e-07, 'band_gap': 0.3954000000000004}
{'material_id': 'mp-19009', 'pretty_formula': 'NiO', 'spg_symprec': 0.1, 'spg_source': 'spglib', 'spg_symbol': 'Fm-3m', 'spg_number': 225, 'poi

In [25]:
row["band_gap"]

0.0

In [20]:
row.data.icsd_ids

[]

In [28]:
my_qs

'X=0,H=0,He=0,Li=0,Be=0,B=0,C=0,N=0,O>=0,F=0,Ne=0,Na=0,Mg=0,Al=0,Si=0,P=0,S=0,Cl=0,Ar=0,K=0,Ca=0,Sc=0,Ti=0,V=0,Cr=0,Mn=0,Fe=0,Co=0,Ni>=0,Cu=0,Zn=0,Ga=0,Ge=0,As=0,Se=0,Br=0,Kr=0,Rb=0,Sr=0,Y=0,Zr=0,Nb=0,Mo=0,Tc=0,Ru=0,Rh=0,Pd=0,Ag=0,Cd=0,In=0,Sn=0,Sb=0,Te=0,I=0,Xe=0,Cs=0,Ba=0,La=0,Ce=0,Pr=0,Nd=0,Pm=0,Sm=0,Eu=0,Gd=0,Tb=0,Dy=0,Ho=0,Er=0,Tm=0,Yb=0,Lu=0,Hf=0,Ta=0,W=0,Re=0,Os=0,Ir=0,Pt=0,Au=0,Hg=0,Tl=0,Pb=0,Bi=0,Po=0,At=0,Rn=0,Fr=0,Ra=0,Ac=0,Th=0,Pa=0,U=0,Np=0,Pu=0,Am=0,Cm=0,Bk=0,Cf=0,Es=0,Fm=0,Md=0,No=0,Lr=0,Rf=0,Db=0,Sg=0,Bh=0,Hs=0,Mt=0,Ds=0,Rg=0,Cn=0,Nh=0,Fl=0,Mc=0,Lv=0,Ts=0,Og=0,,e_above_hull=0'

In [27]:
howmany = 0
for db in dbs:
    for row in db.select(pretty_formula = "NiO"):
        atoms = row.toatoms()
        if len(set(atoms.get_chemical_symbols())) == 1:
            continue
        print(row.key_value_pairs)
        howmany +=1

{'material_id': 'mp-1180047', 'pretty_formula': 'NiO', 'spg_symprec': 0.1, 'spg_source': 'spglib', 'spg_symbol': 'P6/mmm', 'spg_number': 191, 'point_group': '# 6/mmm', 'spg_crystal_system': 'hexagonal', 'spg_hall': '-P 6 2', 'e_above_hull': 1.3137925625000006, 'formation_energy_per_atom': 0.09570726250000039, 'energy_per_atom': -3.65434158, 'volume': 25.502259160055118, 'total_magnetization': 1.9985081, 'band_gap': 0.0}
{'material_id': 'mp-19009', 'pretty_formula': 'NiO', 'spg_symprec': 0.1, 'spg_source': 'spglib', 'spg_symbol': 'Fm-3m', 'spg_number': 225, 'point_group': '# -3m', 'spg_crystal_system': 'cubic', 'spg_hall': '-F 4 2 3', 'e_above_hull': 0, 'formation_energy_per_atom': -1.2180853000000003, 'energy_per_atom': -4.9681341425, 'volume': 37.4854388320674, 'total_magnetization': 0.0, 'band_gap': 2.4141000000000004}
{'e_above_hull': 0.0021408199999999322, 'formation_energy_per_atom': -1.23937222615822, 'material_id': 'oqmd-2580', 'spg_number': 225, 'spg_symbol': 'Fm-3m', 'band_gap